In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
import numpy as np
import cv2
import os,sys 
from glob import glob 

In [2]:
sys.path.append(os.getcwd())
from data import DataLoader

In [3]:
data_dir = 'your_data_path'
train_dir = os.path.join(data_dir,'train')   # your train dataset
test_dir = os.path.join(data_dir,'test')     # your test dataset
txt_path = os.path.join(data_dir,'data.txt') # your label

In [4]:
epochs = 80
batch_size = 2
img_fmt = 'png'
train_loader = DataLoader(
    train_dir,
    txt_path,
    epochs=epochs,
    batch_size = batch_size,
    shuffle_size=batch_size*16,
    img_format=img_fmt
)
test_loader = DataLoader(
    test_dir,
    txt_path,
    img_format=img_fmt,
)
train_data = train_loader.get_dataset().as_numpy_iterator()
test_data = test_loader.get_one_epoch()
img_shape = train_loader.get_img_shape()
f=open("../det0422G.txt","a+")
for(x,y) in test_data:
    f.write (str(y)+'\n')
f.close()

'f=open("../det0422G.txt","a+")\nfor(x,y) in test_data:\n    f.write (str(y)+\'\n\')\nf.close()'

In [5]:
num_output = 4 
dense_model = keras.applications.densenet.DenseNet201(
    include_top = False,
    weights = 'imagenet',
    input_shape = img_shape
)
inputs = keras.Input(shape=img_shape)
#inputs = Input(shape=[img_shape, img_shape, 1])
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv4)
drop4 = Dropout(0.01)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same')(conv5)
drop5 = Dropout(0.01)(conv5)
up6 = Conv2D(512, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv6)
up7 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv7)
up8 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv8)
up9 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv9)
conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same')(conv9)

x = dense_model(conv9) # base_model
x = BatchNormalization()(x)
x = MaxPool2D(3, 2, padding='valid')(x)
x = Flatten()(x)
x = Dropout(0.08)(x)
x = Dense(2048)(x)
x = keras.layers.LeakyReLU(alpha=0.005)(x)
outputs = Dense(num_output)(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [6]:
def acc_reg(y_true, y_pred): # Customize the accuracy function
    mask = tf.less(tf.abs(y_true-y_pred), 0.005)
    num_correct = tf.reduce_sum(tf.cast(mask,tf.int32))
    num_true = tf.reduce_sum(tf.ones_like(y_true))
    acc = tf.cast(num_correct,tf.float32)/(num_true+1e-10)
    return acc

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-7), #1e-5 Init   #5e-5 sec.  #from 1e-6

    loss=tf.keras.losses.Huber(delta=0.0001), #from 0.01 

    metrics= [acc_reg],
)

In [8]:
logdir = "your_save_path" 
if os.path.exists(os.path.join(logdir,'model.h5')):
    print("weights loaded!")
    model.load_weights(os.path.join(logdir,'model.h5'),by_name=True)
    model.trainable = True
    history = model.fit(
    train_data,
    steps_per_epoch=train_loader.get_steps_per_epoch(),
    epochs=epochs,
    validation_data=test_data,
    validation_steps=test_loader.get_steps_per_epoch(),
    callbacks=[
        keras.callbacks.TensorBoard(logdir),
        keras.callbacks.ModelCheckpoint(os.path.join(logdir,'model.h5'),save_best_only=True), # 注意不要直接用logdir
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=5, min_lr=1e-10),
        keras.callbacks.EarlyStopping(patience=5,min_delta=1e-12)
    ]
)

weights loaded!


"history = model.fit(\n    train_data,\n    steps_per_epoch=train_loader.get_steps_per_epoch(),\n    epochs=epochs,\n    validation_data=test_data,\n    validation_steps=test_loader.get_steps_per_epoch(),\n    # 在训练过程（model.fit）添加回调函数,很重要！\n    callbacks=[\n        keras.callbacks.TensorBoard(logdir),\n        keras.callbacks.ModelCheckpoint(os.path.join(logdir,'model.h5'),save_best_only=True), # 注意不要直接用logdir\n        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=5, min_lr=1e-10),\n        keras.callbacks.EarlyStopping(patience=5,min_delta=1e-12)\n    ]\n)"

In [9]:
model.evaluate(test_data)

   2020/Unknown - 140s 69ms/step - loss: 5.4433e-07 - acc_reg: 0.5321

[5.443318973192109e-07, 0.5320545]

In [10]:
predictions = model.predict(test_data)
print(predictions)
np.savetxt("predictions.txt", predictions ,fmt='%f',delimiter=' ')


[[-1.857291  -1.824715   1.6188257 -1.2070547]
 [ 0.9909333 -1.2981311  1.4091825 -1.8178972]
 [-1.6740856  1.7448614  1.8945715  1.0385909]
 ...
 [ 1.1739665  1.7655907  1.4042888  1.2484208]
 [ 1.6524274 -1.9004048 -1.1561234 -1.0292758]
 [ 1.2520217  1.943484   1.0645634  1.9539323]]


In [ ]:
plt.plot(history.epoch, history.history.get('loss'))
plt.plot(history.epoch, history.history.get('val_loss'))
plt.pause(3)

In [ ]:
plt.plot(history.epoch, history.history.get('acc_reg'))
plt.plot(history.epoch, history.history.get('val_acc_reg'))
plt.pause(3)

In [ ]:
predictions = model.predict(test_data)
test_step = min(1,test_loader.get_steps_per_epoch())
for i in range(test_step): 
    print(predictions[i],'\n',np.array(list(test_data)[i][1][0]))